Import libraries

In [1]:
import pandas as pd
import requests
from datetime import datetime
import time
import os
import json
import numpy as np

In [2]:
# set paths and folders
data_path = '../../Data/20230414DataJobSearch/'
folders = ['2023-04-14-job-search-location-USA', '2023-04-14-job-search-location-DC']


### Metadata and parameters

In [3]:
# init empty list
metadata_and_parameters = []

# loop through all files
for folder in folders:
    
    data_path_and_folder = f'{data_path}{folder}'
    
    for file in os.listdir(data_path_and_folder):
        if file.endswith(".json"):
            full_path = os.path.join(data_path_and_folder, file)
            
            # open json
            with open(full_path, 'r') as f:
                json_data = json.load(f)
            
            # store file and folder name
            json_data['search_name'] = file
            json_data['search_location'] = folder
            
            # append
            metadata_and_parameters.append(json_data)

# convert to dataframe
metadata_and_parameters = pd.json_normalize(metadata_and_parameters)

# drop results and chips
metadata_and_parameters = metadata_and_parameters.drop(['jobs_results', 'chips'], axis=1)

# clean search name and location
metadata_and_parameters['search_name'] = metadata_and_parameters['search_name'].str.replace('.json', '', regex=False)
metadata_and_parameters['search_location'] = metadata_and_parameters['search_location'].str.replace('2023-04-14-job-search-location-', '', regex=False)

# fix column names
metadata_and_parameters = metadata_and_parameters.rename(columns=lambda x: x.replace('search_metadata.', 'metadata_'))
metadata_and_parameters = metadata_and_parameters.rename(columns=lambda x: x.replace('search_parameters.', 'parameters_'))

# check
display(metadata_and_parameters.shape)
display(metadata_and_parameters.head())



(85, 19)

,search_name,search_location,metadata_id,metadata_status,metadata_json_endpoint,metadata_created_at,metadata_processed_at,metadata_google_jobs_url,metadata_raw_html_file,metadata_total_time_taken,parameters_q,parameters_engine,parameters_uule,parameters_google_domain,parameters_hl,parameters_gl,parameters_start,error,search_information.jobs_results_state
0,block-chain-1,USA,643db9ec224db12213cd421b,Success,https://serpapi.com/searches/a7e3d320a9075f00/...,2023-04-17 21:28:12 UTC,2023-04-17 21:28:12 UTC,https://www.google.com/search?q=block+chain&ib...,https://serpapi.com/searches/a7e3d320a9075f00/...,4.09,block chain,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,10,NaN,NaN
1,data-scientist-4,USA,643db97e5fc493f4ef82a469,Success,https://serpapi.com/searches/a980517c7e93b15a/...,2023-04-17 21:26:22 UTC,2023-04-17 21:26:22 UTC,https://www.google.com/search?q=data+scientist...,https://serpapi.com/searches/a980517c7e93b15a/...,1.84,data scientist,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,40,NaN,NaN
2,natural-language-processing-1,USA,643db9f96073930b48d5359a,Success,https://serpapi.com/searches/6806a6ad5db11221/...,2023-04-17 21:28:25 UTC,2023-04-17 21:28:25 UTC,https://www.google.com/search?q=natural+langua...,https://serpapi.com/searches/6806a6ad5db11221/...,2.50,natural language processing,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,10,NaN,NaN
3,reinforcement-learning-4,USA,643db9ce58762b07c185c1b1,Success,https://serpapi.com/searches/4355f1f2ab30af9b/...,2023-04-17 21:27:42 UTC,2023-04-17 21:27:42 UTC,https://www.google.com/search?q=reinforcement+...,https://serpapi.com/searches/4355f1f2ab30af9b/...,2.69,reinforcement learning,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,40,NaN,NaN
4,neural-networks-4,USA,643db99ab7b1cc54dd45c12f,Success,https://serpapi.com/searches/1260623261e4e9be/...,2023-04-17 21:26:50 UTC,2023-04-17 21:26:50 UTC,https://www.google.com/search?q=neural+network...,https://serpapi.com/searches/1260623261e4e9be/...,0.69,neural networks,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,40,NaN,NaN


In [4]:
# remove searches that failed
metadata_and_parameters = metadata_and_parameters[~metadata_and_parameters['search_information.jobs_results_state'].isin(['Fully empty'])]

# drop columns that indicate failed search
metadata_and_parameters = metadata_and_parameters.drop(['error', 'search_information.jobs_results_state'], axis=1)

# remove duplicates if any
metadata_and_parameters = metadata_and_parameters.drop_duplicates()

# check
display(metadata_and_parameters.shape)
display(metadata_and_parameters.head())


(83, 17)

,search_name,search_location,metadata_id,metadata_status,metadata_json_endpoint,metadata_created_at,metadata_processed_at,metadata_google_jobs_url,metadata_raw_html_file,metadata_total_time_taken,parameters_q,parameters_engine,parameters_uule,parameters_google_domain,parameters_hl,parameters_gl,parameters_start
0,block-chain-1,USA,643db9ec224db12213cd421b,Success,https://serpapi.com/searches/a7e3d320a9075f00/...,2023-04-17 21:28:12 UTC,2023-04-17 21:28:12 UTC,https://www.google.com/search?q=block+chain&ib...,https://serpapi.com/searches/a7e3d320a9075f00/...,4.09,block chain,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,10
1,data-scientist-4,USA,643db97e5fc493f4ef82a469,Success,https://serpapi.com/searches/a980517c7e93b15a/...,2023-04-17 21:26:22 UTC,2023-04-17 21:26:22 UTC,https://www.google.com/search?q=data+scientist...,https://serpapi.com/searches/a980517c7e93b15a/...,1.84,data scientist,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,40
2,natural-language-processing-1,USA,643db9f96073930b48d5359a,Success,https://serpapi.com/searches/6806a6ad5db11221/...,2023-04-17 21:28:25 UTC,2023-04-17 21:28:25 UTC,https://www.google.com/search?q=natural+langua...,https://serpapi.com/searches/6806a6ad5db11221/...,2.50,natural language processing,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,10
3,reinforcement-learning-4,USA,643db9ce58762b07c185c1b1,Success,https://serpapi.com/searches/4355f1f2ab30af9b/...,2023-04-17 21:27:42 UTC,2023-04-17 21:27:42 UTC,https://www.google.com/search?q=reinforcement+...,https://serpapi.com/searches/4355f1f2ab30af9b/...,2.69,reinforcement learning,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,40
4,neural-networks-4,USA,643db99ab7b1cc54dd45c12f,Success,https://serpapi.com/searches/1260623261e4e9be/...,2023-04-17 21:26:50 UTC,2023-04-17 21:26:50 UTC,https://www.google.com/search?q=neural+network...,https://serpapi.com/searches/1260623261e4e9be/...,0.69,neural networks,google_jobs,w+CAIQICINVW5pdGVkIFN0YXRlcw,google.com,en,us,40


In [5]:
# export
metadata_and_parameters.to_csv('../../Data/metadata_and_parameters.csv', index=False)


### Job Results

In [6]:
# init empty list
jobs_results = []

# loop through all files
for folder in folders:
    data_path_and_folder = os.path.join(data_path, folder)

    for file in os.listdir(data_path_and_folder):
        
        if True == False: # set to true to debug
            print(folder)
            print(file)
        
        
        if file.endswith(".json"):
            full_path = os.path.join(data_path_and_folder, file)

            # open json
            with open(full_path, 'r') as f:
                json_data = json.load(f)
            
            # check if the file has failed search columns and skip if yes
            try:
                json_data["search_information"]["jobs_results_state"]                
                continue
            
            except:       
                pass
                

            # select only job results
            json_data = json_data["jobs_results"]

            
            for job in json_data:
                highlights_dict = {}
                
                # create job highlight cols
                for highlight in job["job_highlights"]:
                    if "title" in highlight and "items" in highlight: # check if there are highlights with titles, some don't have it
                        highlights_dict[highlight["title"]] = "\n".join(highlight["items"])

                jobs_results.append({
                    "title": job["title"],
                    "company_name": job["company_name"],
                    "location": job["location"].strip(),
                    "via": job["via"],
                    "description": job["description"],
                    "qualifications": highlights_dict.get("Qualifications", ""),
                    "responsibilities": highlights_dict.get("Responsibilities", ""),
                    "benefits": highlights_dict.get("Benefits", ""),
                    #"extensions": job["extensions"],
                    "detected_extensions": job["detected_extensions"],
                    "job_id": job["job_id"],
                    "search_name": file,
                    "search_location": folder
                })
                
            
# convert to dataframe
jobs_results = pd.DataFrame(jobs_results)

# clean search name and location
jobs_results['search_name'] = jobs_results['search_name'].str.replace('.json', '', regex=False)
jobs_results['search_location'] = jobs_results['search_location'].str.replace('2023-04-14-job-search-location-', '', regex=False)

# check
display(jobs_results.shape)
display(jobs_results.head())


(823, 12)

,title,company_name,location,via,description,qualifications,responsibilities,benefits,detected_extensions,job_id,search_name,search_location
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,"{'posted_at': '7 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,,,,"{'posted_at': '4 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,{'schedule_type': 'Full-time'},eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",{'schedule_type': 'Full-time'},eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,,,,"{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA


In [7]:
# expand the detected extensions col

# get all possible keys
detected_extensions_keys = set()
for extensions in jobs_results['detected_extensions']:
    
    detected_extensions_keys.update(extensions.keys())

# create a new df
detected_extensions = []
for i in jobs_results['detected_extensions']:
    
    ext_dict = {}
    for key in detected_extensions_keys:
        ext_dict[key] = i.get(key, np.nan) # get key or nan otherwise
        
    detected_extensions.append(ext_dict)

detected_extensions = pd.DataFrame(detected_extensions)


#check
display(detected_extensions.head())

,posted_at,schedule_type,salary,work_from_home,commute_time
0,7 days ago,Full-time,NaN,True,NaN
1,4 days ago,Contractor,NaN,True,NaN
2,NaN,Full-time,NaN,NaN,NaN
3,NaN,Full-time,NaN,NaN,NaN
4,2 days ago,Full-time,20–28 an hour,NaN,NaN


In [8]:
# concat and drop unnecesary column
jobs_results = pd.concat([jobs_results, detected_extensions], axis=1)
jobs_results = jobs_results.drop('detected_extensions', axis=1)

# check
display(jobs_results.shape)
display(jobs_results.head())

(823, 16)

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,search_name,search_location,posted_at,schedule_type,salary,work_from_home,commute_time
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA,7 days ago,Full-time,NaN,True,NaN
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,,,,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA,4 days ago,Contractor,NaN,True,NaN
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA,NaN,Full-time,NaN,NaN,NaN
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA,NaN,Full-time,NaN,NaN,NaN
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,,,,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA,2 days ago,Full-time,20–28 an hour,NaN,NaN


In [9]:
# check missingness
jobs_results.isnull().sum()


title                 0
company_name          0
location              0
via                   0
description           0
qualifications        0
responsibilities      0
benefits              0
job_id                0
search_name           0
search_location       0
posted_at           293
schedule_type         1
salary              690
work_from_home      646
commute_time        822
dtype: int64

In [10]:
# keep only schedule_type
jobs_results = jobs_results.drop(['posted_at', 'work_from_home', 'commute_time', 'salary'], axis=1)

# fill empty shedule type
jobs_results['schedule_type'] = jobs_results['schedule_type'].fillna('Not Specified')


In [11]:
# remove duplicates if any based on job id
jobs_results = jobs_results.drop_duplicates(subset=['title', 'company_name', 'location'])

# check
display(jobs_results.shape)
display(jobs_results.head())

(591, 12)

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,search_name,search_location,schedule_type
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA,Full-time
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,,,,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA,Contractor
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA,Full-time
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA,Full-time
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,,,,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA,Full-time


In [12]:
# confirm no duplicates
jobs_results['job_id'].nunique()


591

In [13]:
# format
jobs_results['category'] = jobs_results['search_name'].str.replace('-', ' ') # replace dashes
jobs_results['category'] = jobs_results['category'].str.title() # capitalize each word
jobs_results['category'] = jobs_results['category'].str.replace('\d+', '', regex=True) # remove the numbers

display(jobs_results.head())


,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,search_name,search_location,schedule_type,category
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA,Full-time,Block Chain
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,,,,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA,Contractor,Block Chain
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA,Full-time,Block Chain
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA,Full-time,Block Chain
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,,,,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA,Full-time,Block Chain


In [14]:
# export
jobs_results.to_csv('../../Data/jobs_results.csv', index=False)